# 🔧 Normalisation Avancée des Entités

**Module 2 : Preprocessing et Tokenisation**

---

## 🎯 Objectifs de ce Notebook

Dans ce notebook, vous allez apprendre à :

- 📅 **Normaliser les dates** : "3 mars 2024", "03/03/2024" → "DATE"
- 💰 **Normaliser les montants** : "1.500,50€", "1500.5 euros" → "MONTANT"
- 📧 **Normaliser les emails** : "contact@exemple.fr" → "EMAIL"
- 🌐 **Normaliser les URLs** : "https://www.exemple.com" → "URL"
- 📱 **Normaliser les téléphones** : "01 23 45 67 89" → "TELEPHONE"
- 🔢 **Normaliser les nombres** : "mille", "1000", "1 000" → "NOMBRE"

## 💡 Pourquoi Normaliser les Entités ?

**Problème :** Dans le texte brut, la même information peut s'écrire de multiples façons :
```
"Contactez-moi au 01.23.45.67.89 ou 01 23 45 67 89 ou +33123456789"
```

**Solution :** Remplacer par un token unique :
```
"Contactez-moi au TELEPHONE ou TELEPHONE ou TELEPHONE"
```

**Avantages :**
- 📉 **Réduction du vocabulaire** : Moins de mots uniques
- 🎯 **Focus sur le sens** : Le modèle se concentre sur le contenu, pas la forme
- 🔒 **Anonymisation** : Protection des données personnelles
- ⚡ **Performance** : Moins de tokens = calculs plus rapides

In [ ]:
# Installation des dépendances (si nécessaire)
# !pip install regex pandas matplotlib

import re
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from typing import Dict, List, Tuple

# Configuration pour l'affichage
plt.style.use('default')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ Imports réussis ! Prêt pour la normalisation avancée.")

---

# 📅 1. Normalisation des Dates

Les dates peuvent s'écrire de nombreuses façons en français :

In [ ]:
# Exemples de dates en français
exemples_dates = [
    "3 mars 2024",
    "15 janvier 2023",
    "03/03/2024",
    "15/01/2023",
    "3 mars",
    "le 25 décembre",
    "2024-03-15",
    "mars 2024",
    "en janvier 2023"
]

print("📅 Exemples de dates à normaliser :")
for i, date in enumerate(exemples_dates, 1):
    print(f"{i:2d}. {date}")

In [ ]:
class NormalisateurDates:
    """Classe pour normaliser les dates en français"""
    
    def __init__(self):
        # Dictionnaire des mois français
        self.mois_francais = {
            'janvier': '01', 'février': '02', 'mars': '03', 'avril': '04',
            'mai': '05', 'juin': '06', 'juillet': '07', 'août': '08',
            'septembre': '09', 'octobre': '10', 'novembre': '11', 'décembre': '12'
        }
        
        # Patterns regex pour différents formats
        self.patterns = [
            # Format: "3 mars 2024"
            r'\b(\d{1,2})\s+(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)\s+(\d{4})\b',
            # Format: "03/03/2024" ou "3/3/2024"
            r'\b(\d{1,2})/(\d{1,2})/(\d{4})\b',
            # Format: "2024-03-15"
            r'\b(\d{4})-(\d{1,2})-(\d{1,2})\b',
            # Format: "le 25 décembre"
            r'\ble\s+(\d{1,2})\s+(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)\b',
        ]
    
    def normaliser_texte(self, texte: str, remplacer_par: str = "DATE") -> str:
        """
        Normalise toutes les dates dans un texte
        
        Args:
            texte: Le texte à traiter
            remplacer_par: Token de remplacement (défaut: "DATE")
        
        Returns:
            Texte avec dates normalisées
        """
        resultat = texte
        
        for pattern in self.patterns:
            resultat = re.sub(pattern, remplacer_par, resultat, flags=re.IGNORECASE)
        
        return resultat
    
    def detecter_dates(self, texte: str) -> List[str]:
        """
        Détecte toutes les dates dans un texte
        
        Returns:
            Liste des dates trouvées
        """
        dates_trouvees = []
        
        for pattern in self.patterns:
            matches = re.findall(pattern, texte, re.IGNORECASE)
            for match in matches:
                if isinstance(match, tuple):
                    dates_trouvees.extend([m for m in match if m])
                else:
                    dates_trouvees.append(match)
        
        return dates_trouvees

# Test du normalisateur de dates
normalisateur_dates = NormalisateurDates()

print("🧪 Test du normalisateur de dates :")
print("=" * 50)

texte_test = "Rendez-vous le 15 mars 2024 à 14h30. Le 03/01/2023 était un mardi."
print(f"📝 Texte original : {texte_test}")
print(f"🔧 Après normalisation : {normalisateur_dates.normaliser_texte(texte_test)}")
print(f"🔍 Dates détectées : {normalisateur_dates.detecter_dates(texte_test)}")

---

# 💰 2. Normalisation des Montants

Les montants peuvent s'écrire de multiples façons :

In [ ]:
# Exemples de montants
exemples_montants = [
    "1.500,50€",
    "1500.5 euros",
    "25€",
    "€25",
    "25 EUR",
    "25 dollars",
    "$25",
    "25$",
    "2 000 €",
    "15,99€",
    "1 million d'euros",
    "vingt euros"
]

print("💰 Exemples de montants à normaliser :")
for i, montant in enumerate(exemples_montants, 1):
    print(f"{i:2d}. {montant}")

In [ ]:
class NormalisateurMontants:
    """Classe pour normaliser les montants"""
    
    def __init__(self):
        # Patterns pour différents formats de montants
        self.patterns = [
            # Format français: "1.500,50€" ou "1 500,50 €"
            r'\b\d{1,3}(?:[.\s]\d{3})*,\d{2}\s*€?\b',
            # Format anglais: "1,500.50" ou "$1500.50"
            r'\$?\b\d{1,3}(?:,\d{3})*\.\d{2}\b',
            # Montants simples: "25€", "€25", "25 euros"
            r'\b\d+(?:\.\d+)?\s*(?:€|euros?|EUR|dollars?|\$)\b',
            r'\b(?:€|\$)\d+(?:\.\d+)?\b',
            # Montants en lettres approximatifs
            r'\b(?:un|deux|trois|quatre|cinq|six|sept|huit|neuf|dix|vingt|trente|quarante|cinquante|cent|mille|million)\s+(?:euros?|dollars?)\b',
        ]
    
    def normaliser_texte(self, texte: str, remplacer_par: str = "MONTANT") -> str:
        """
        Normalise tous les montants dans un texte
        """
        resultat = texte
        
        for pattern in self.patterns:
            resultat = re.sub(pattern, remplacer_par, resultat, flags=re.IGNORECASE)
        
        return resultat
    
    def detecter_montants(self, texte: str) -> List[str]:
        """
        Détecte tous les montants dans un texte
        """
        montants_trouves = []
        
        for pattern in self.patterns:
            matches = re.findall(pattern, texte, re.IGNORECASE)
            montants_trouves.extend(matches)
        
        return montants_trouves

# Test du normalisateur de montants
normalisateur_montants = NormalisateurMontants()

print("🧪 Test du normalisateur de montants :")
print("=" * 50)

texte_test = "Le produit coûte 1.299,99€ en promo au lieu de 1500 euros. Livraison : 5,99€."
print(f"📝 Texte original : {texte_test}")
print(f"🔧 Après normalisation : {normalisateur_montants.normaliser_texte(texte_test)}")
print(f"🔍 Montants détectés : {normalisateur_montants.detecter_montants(texte_test)}")

---

# 📧 3. Normalisation des Emails et URLs

Emails et URLs sont facilement détectables avec des regex :

In [ ]:
class NormalisateurWeb:
    """Classe pour normaliser emails, URLs et mentions"""
    
    def __init__(self):
        # Patterns pour les éléments web
        self.patterns = {
            'email': r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',
            'url': r'https?://[^\s<>"\[\]{}|\\^`]+',
            'url_www': r'www\.[^\s<>"\[\]{}|\\^`]+',
            'mention': r'@\w+',
            'hashtag': r'#\w+'
        }
    
    def normaliser_texte(self, texte: str, 
                        remplacements: Dict[str, str] = None) -> str:
        """
        Normalise les éléments web dans un texte
        
        Args:
            texte: Texte à traiter
            remplacements: Dictionnaire des remplacements personnalisés
        """
        if remplacements is None:
            remplacements = {
                'email': 'EMAIL',
                'url': 'URL',
                'url_www': 'URL',
                'mention': 'MENTION',
                'hashtag': 'HASHTAG'
            }
        
        resultat = texte
        
        for type_element, pattern in self.patterns.items():
            if type_element in remplacements:
                resultat = re.sub(pattern, remplacements[type_element], resultat)
        
        return resultat
    
    def detecter_elements(self, texte: str) -> Dict[str, List[str]]:
        """
        Détecte tous les éléments web dans un texte
        """
        elements_trouves = {}
        
        for type_element, pattern in self.patterns.items():
            matches = re.findall(pattern, texte)
            if matches:
                elements_trouves[type_element] = matches
        
        return elements_trouves

# Test du normalisateur web
normalisateur_web = NormalisateurWeb()

print("🧪 Test du normalisateur web :")
print("=" * 50)

texte_test = "Contactez-moi à jean.dupont@example.fr ou sur https://example.com. Suivez @moncompte #nlp"
print(f"📝 Texte original : {texte_test}")
print(f"🔧 Après normalisation : {normalisateur_web.normaliser_texte(texte_test)}")
print(f"🔍 Éléments détectés : {normalisateur_web.detecter_elements(texte_test)}")

---

# 📱 4. Normalisation des Téléphones

Les numéros de téléphone français ont de nombreux formats :

In [ ]:
# Exemples de numéros de téléphone français
exemples_telephones = [
    "01 23 45 67 89",
    "01.23.45.67.89",
    "01-23-45-67-89",
    "0123456789",
    "+33 1 23 45 67 89",
    "+33123456789",
    "06 12 34 56 78",  # Mobile
    "09 12 34 56 78",  # Numéro spéciaux
    "0800 12 34 56"    # Numéro vert
]

print("📱 Exemples de téléphones français :")
for i, tel in enumerate(exemples_telephones, 1):
    print(f"{i:2d}. {tel}")

In [ ]:
class NormalisateurTelephones:
    """Classe pour normaliser les numéros de téléphone français"""
    
    def __init__(self):
        # Patterns pour les téléphones français
        self.patterns = [
            # Format international: "+33 1 23 45 67 89"
            r'\+33\s?[1-9](?:[\s.-]?\d{2}){4}\b',
            # Format national avec espaces: "01 23 45 67 89"
            r'\b0[1-9](?:\s\d{2}){4}\b',
            # Format avec points: "01.23.45.67.89"
            r'\b0[1-9](?:\.\d{2}){4}\b',
            # Format avec tirets: "01-23-45-67-89"
            r'\b0[1-9](?:-\d{2}){4}\b',
            # Format compact: "0123456789"
            r'\b0[1-9]\d{8}\b',
            # Numéros verts et spéciaux
            r'\b0800\s?\d{2}\s?\d{2}\s?\d{2}\b'
        ]
    
    def normaliser_texte(self, texte: str, remplacer_par: str = "TELEPHONE") -> str:
        """
        Normalise tous les téléphones dans un texte
        """
        resultat = texte
        
        for pattern in self.patterns:
            resultat = re.sub(pattern, remplacer_par, resultat)
        
        return resultat
    
    def detecter_telephones(self, texte: str) -> List[str]:
        """
        Détecte tous les téléphones dans un texte
        """
        telephones_trouves = []
        
        for pattern in self.patterns:
            matches = re.findall(pattern, texte)
            telephones_trouves.extend(matches)
        
        return telephones_trouves
    
    def valider_telephone(self, telephone: str) -> bool:
        """
        Valide si un téléphone français est correct
        """
        # Nettoyer le numéro
        numero_propre = re.sub(r'[^\d+]', '', telephone)
        
        # Vérifications basiques
        if numero_propre.startswith('+33'):
            return len(numero_propre) == 12 and numero_propre[3] in '123456789'
        elif numero_propre.startswith('0'):
            return len(numero_propre) == 10 and numero_propre[1] in '123456789'
        
        return False

# Test du normalisateur de téléphones
normalisateur_tel = NormalisateurTelephones()

print("🧪 Test du normalisateur de téléphones :")
print("=" * 50)

texte_test = "Appelez-moi au 01.23.45.67.89 ou au +33 6 12 34 56 78 après 18h."
print(f"📝 Texte original : {texte_test}")
print(f"🔧 Après normalisation : {normalisateur_tel.normaliser_texte(texte_test)}")
print(f"🔍 Téléphones détectés : {normalisateur_tel.detecter_telephones(texte_test)}")

# Test de validation
print("\n✅ Tests de validation :")
for tel in exemples_telephones[:5]:
    valide = normalisateur_tel.valider_telephone(tel)
    print(f"  {tel:<20} → {'✅ Valide' if valide else '❌ Invalide'}")

---

# 🔢 5. Normalisation des Nombres

Les nombres peuvent être écrits en chiffres ou en lettres :

In [ ]:
class NormalisateurNombres:
    """Classe pour normaliser les nombres"""
    
    def __init__(self):
        # Dictionnaire des nombres en lettres
        self.nombres_lettres = {
            'zéro': '0', 'un': '1', 'deux': '2', 'trois': '3', 'quatre': '4',
            'cinq': '5', 'six': '6', 'sept': '7', 'huit': '8', 'neuf': '9',
            'dix': '10', 'onze': '11', 'douze': '12', 'treize': '13',
            'quatorze': '14', 'quinze': '15', 'seize': '16',
            'vingt': '20', 'trente': '30', 'quarante': '40',
            'cinquante': '50', 'soixante': '60', 'soixante-dix': '70',
            'quatre-vingts': '80', 'quatre-vingt-dix': '90',
            'cent': '100', 'mille': '1000', 'million': '1000000'
        }
        
        # Patterns pour différents formats de nombres
        self.patterns = [
            # Nombres avec séparateurs: "1 000", "1.000", "1,000"
            r'\b\d{1,3}(?:[.\s,]\d{3})+\b',
            # Nombres décimaux: "1.5", "1,5"
            r'\b\d+[.,]\d+\b',
            # Nombres simples: "123"
            r'\b\d+\b',
        ]
    
    def normaliser_nombres_lettres(self, texte: str) -> str:
        """
        Convertit les nombres en lettres en chiffres
        """
        resultat = texte.lower()
        
        # Remplacer les nombres en lettres par leurs équivalents chiffrés
        for mot, chiffre in self.nombres_lettres.items():
            pattern = r'\b' + re.escape(mot) + r'\b'
            resultat = re.sub(pattern, chiffre, resultat)
        
        return resultat
    
    def normaliser_texte(self, texte: str, 
                        remplacer_par: str = "NOMBRE",
                        convertir_lettres: bool = True) -> str:
        """
        Normalise tous les nombres dans un texte
        """
        resultat = texte
        
        # Convertir d'abord les nombres en lettres
        if convertir_lettres:
            resultat = self.normaliser_nombres_lettres(resultat)
        
        # Remplacer tous les nombres par le token
        for pattern in self.patterns:
            resultat = re.sub(pattern, remplacer_par, resultat)
        
        return resultat
    
    def detecter_nombres(self, texte: str) -> Dict[str, List[str]]:
        """
        Détecte tous les nombres dans un texte
        """
        nombres_trouves = {'chiffres': [], 'lettres': []}
        
        # Détecter les nombres en chiffres
        for pattern in self.patterns:
            matches = re.findall(pattern, texte)
            nombres_trouves['chiffres'].extend(matches)
        
        # Détecter les nombres en lettres
        for mot in self.nombres_lettres.keys():
            pattern = r'\b' + re.escape(mot) + r'\b'
            if re.search(pattern, texte.lower()):
                nombres_trouves['lettres'].append(mot)
        
        return nombres_trouves

# Test du normalisateur de nombres
normalisateur_nombres = NormalisateurNombres()

print("🧪 Test du normalisateur de nombres :")
print("=" * 50)

texte_test = "J'ai acheté trois produits pour 1.250 euros, soit environ mille euros chacun."
print(f"📝 Texte original : {texte_test}")
print(f"🔧 Après conversion lettres→chiffres : {normalisateur_nombres.normaliser_nombres_lettres(texte_test)}")
print(f"🔧 Après normalisation complète : {normalisateur_nombres.normaliser_texte(texte_test)}")
print(f"🔍 Nombres détectés : {normalisateur_nombres.detecter_nombres(texte_test)}")

---

# 🛠️ 6. Normalisateur Complet

Créons maintenant une classe qui combine toutes les normalisations :

In [ ]:
class NormalisateurComplet:
    """Normalisateur qui combine toutes les techniques"""
    
    def __init__(self, 
                 normaliser_dates: bool = True,
                 normaliser_montants: bool = True,
                 normaliser_web: bool = True,
                 normaliser_telephones: bool = True,
                 normaliser_nombres: bool = True):
        """
        Initialise le normalisateur avec les options choisies
        """
        self.config = {
            'dates': normaliser_dates,
            'montants': normaliser_montants,
            'web': normaliser_web,
            'telephones': normaliser_telephones,
            'nombres': normaliser_nombres
        }
        
        # Initialiser les normalisateurs spécialisés
        if self.config['dates']:
            self.norm_dates = NormalisateurDates()
        if self.config['montants']:
            self.norm_montants = NormalisateurMontants()
        if self.config['web']:
            self.norm_web = NormalisateurWeb()
        if self.config['telephones']:
            self.norm_tel = NormalisateurTelephones()
        if self.config['nombres']:
            self.norm_nombres = NormalisateurNombres()
    
    def normaliser_texte(self, texte: str, verbose: bool = False) -> str:
        """
        Applique toutes les normalisations configurées
        
        Args:
            texte: Texte à normaliser
            verbose: Affiche les étapes si True
        
        Returns:
            Texte normalisé
        """
        resultat = texte
        etapes = [f"📝 Original: {resultat}"]
        
        # Appliquer les normalisations dans l'ordre
        if self.config['dates']:
            resultat = self.norm_dates.normaliser_texte(resultat)
            etapes.append(f"📅 Dates: {resultat}")
        
        if self.config['montants']:
            resultat = self.norm_montants.normaliser_texte(resultat)
            etapes.append(f"💰 Montants: {resultat}")
        
        if self.config['telephones']:
            resultat = self.norm_tel.normaliser_texte(resultat)
            etapes.append(f"📱 Téléphones: {resultat}")
        
        if self.config['web']:
            resultat = self.norm_web.normaliser_texte(resultat)
            etapes.append(f"🌐 Web: {resultat}")
        
        if self.config['nombres']:
            resultat = self.norm_nombres.normaliser_texte(resultat)
            etapes.append(f"🔢 Nombres: {resultat}")
        
        if verbose:
            for etape in etapes:
                print(etape)
        
        return resultat
    
    def analyser_texte(self, texte: str) -> Dict:
        """
        Analyse complète d'un texte
        
        Returns:
            Dictionnaire avec statistiques et éléments détectés
        """
        analyse = {
            'texte_original': texte,
            'texte_normalise': self.normaliser_texte(texte),
            'statistiques': {},
            'elements_detectes': {}
        }
        
        # Compter les mots avant/après
        mots_avant = len(texte.split())
        mots_apres = len(analyse['texte_normalise'].split())
        
        analyse['statistiques'] = {
            'mots_avant': mots_avant,
            'mots_apres': mots_apres,
            'reduction_mots': mots_avant - mots_apres,
            'pourcentage_reduction': round((mots_avant - mots_apres) / mots_avant * 100, 1) if mots_avant > 0 else 0
        }
        
        # Détecter les éléments
        if self.config['dates']:
            analyse['elements_detectes']['dates'] = self.norm_dates.detecter_dates(texte)
        if self.config['montants']:
            analyse['elements_detectes']['montants'] = self.norm_montants.detecter_montants(texte)
        if self.config['web']:
            analyse['elements_detectes']['web'] = self.norm_web.detecter_elements(texte)
        if self.config['telephones']:
            analyse['elements_detectes']['telephones'] = self.norm_tel.detecter_telephones(texte)
        if self.config['nombres']:
            analyse['elements_detectes']['nombres'] = self.norm_nombres.detecter_nombres(texte)
        
        return analyse

# Test du normalisateur complet
normalisateur_complet = NormalisateurComplet()

print("🚀 Test du normalisateur complet :")
print("=" * 70)

---

# 🧪 7. Exercices Pratiques

Testons le normalisateur sur des exemples réels :

In [ ]:
# Exemples de textes réels à normaliser
textes_exemples = [
    "Rendez-vous le 15 mars 2024 à 14h30. Contactez-moi au 01.23.45.67.89 ou jean.dupont@email.fr. Budget : 1.500,50€.",
    "Super promotion ! -50% sur https://boutique.com jusqu'au 31/12/2023. Livraison gratuite dès 50€. #promo @followers",
    "Facture n°123 du 01/01/2024. Montant TTC : deux mille euros. Contact : +33 6 12 34 56 78.",
    "Voir www.example.com pour plus d'infos. Prix : $99.99 ou 89,99€. Tél : 0800 12 34 56."
]

print("🎯 EXERCICES PRATIQUES")
print("=" * 50)

for i, texte in enumerate(textes_exemples, 1):
    print(f"\n📝 **Exemple {i}:**")
    print(f"Original : {texte}")
    
    # Analyse complète
    analyse = normalisateur_complet.analyser_texte(texte)
    
    print(f"Normalisé : {analyse['texte_normalise']}")
    print(f"📊 Stats : {analyse['statistiques']['reduction_mots']} mots supprimés ({analyse['statistiques']['pourcentage_reduction']}%)")
    
    # Afficher les éléments détectés
    elements = analyse['elements_detectes']
    if any(elements.values()):
        print("🔍 Éléments détectés :")
        for type_elem, contenu in elements.items():
            if contenu:
                print(f"  • {type_elem}: {contenu}")

## 🎯 Exercice Interactif

À votre tour ! Testez le normalisateur sur vos propres textes :

In [ ]:
# 🎯 EXERCICE INTERACTIF
# Modifiez ce texte avec vos propres exemples

mon_texte = """
Écrivez ici votre texte à normaliser...
Exemples d'éléments à inclure :
- Dates : le 25 décembre 2024
- Montants : 99,99€ ou vingt euros
- Emails : test@example.com
- URLs : https://example.com
- Téléphones : 01 23 45 67 89
- Nombres : mille ou 1000
"""

# Configuration personnalisée du normalisateur
mon_normalisateur = NormalisateurComplet(
    normaliser_dates=True,
    normaliser_montants=True,
    normaliser_web=True,
    normaliser_telephones=True,
    normaliser_nombres=True
)

# Test avec affichage des étapes
print("🔍 VOTRE TEST PERSONNALISÉ :")
print("=" * 50)
resultat = mon_normalisateur.normaliser_texte(mon_texte.strip(), verbose=True)

print(f"\n✅ **Résultat final :**\n{resultat}")

---

# 📊 8. Analyse Comparative

Comparons l'impact des différentes normalisations :

In [ ]:
# Créer différentes configurations de normalisateurs
configs = {
    'Aucune': NormalisateurComplet(False, False, False, False, False),
    'Dates seulement': NormalisateurComplet(True, False, False, False, False),
    'Web seulement': NormalisateurComplet(False, False, True, False, False),
    'Montants + Nombres': NormalisateurComplet(False, True, False, False, True),
    'Complète': NormalisateurComplet(True, True, True, True, True)
}

# Texte de test complexe
texte_test_complexe = """
Bonjour ! Rendez-vous le 15 mars 2024 à 14h30 pour discuter du budget de mille euros.
Contactez-moi à marie.dupont@entreprise.fr ou au 01.23.45.67.89.
Voir aussi notre site https://entreprise.com pour plus d'infos.
Promotion : -20% soit 199,99€ au lieu de 250€ jusqu'au 31/12/2023.
Suivez-nous @entreprise #innovation
""".strip()

print("📊 ANALYSE COMPARATIVE DES CONFIGURATIONS")
print("=" * 60)
print(f"📝 Texte original ({len(texte_test_complexe.split())} mots) :")
print(texte_test_complexe)
print()

# Tableau comparatif
resultats = []

for nom_config, normalisateur in configs.items():
    analyse = normalisateur.analyser_texte(texte_test_complexe)
    
    resultats.append({
        'Configuration': nom_config,
        'Mots avant': analyse['statistiques']['mots_avant'],
        'Mots après': analyse['statistiques']['mots_apres'],
        'Réduction': analyse['statistiques']['reduction_mots'],
        '% Réduction': analyse['statistiques']['pourcentage_reduction']
    })
    
    print(f"🔧 **{nom_config}** ({analyse['statistiques']['mots_apres']} mots, -{analyse['statistiques']['pourcentage_reduction']}%) :")
    print(f"   {analyse['texte_normalise']}")
    print()

# Créer un DataFrame pour l'analyse
df_resultats = pd.DataFrame(resultats)
print("📈 **TABLEAU RÉCAPITULATIF :**")
print(df_resultats.to_string(index=False))

In [ ]:
# Visualisation des résultats
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Graphique 1: Nombre de mots
configs_noms = df_resultats['Configuration']
mots_avant = df_resultats['Mots avant']
mots_apres = df_resultats['Mots après']

x = range(len(configs_noms))
width = 0.35

ax1.bar([i - width/2 for i in x], mots_avant, width, label='Avant', color='lightcoral', alpha=0.8)
ax1.bar([i + width/2 for i in x], mots_apres, width, label='Après', color='lightblue', alpha=0.8)

ax1.set_xlabel('Configuration')
ax1.set_ylabel('Nombre de mots')
ax1.set_title('📊 Nombre de mots avant/après normalisation')
ax1.set_xticks(x)
ax1.set_xticklabels(configs_noms, rotation=45, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Graphique 2: Pourcentage de réduction
pourcentages = df_resultats['% Réduction']
colors = plt.cm.viridis(np.linspace(0, 1, len(pourcentages)))

bars = ax2.bar(configs_noms, pourcentages, color=colors, alpha=0.8)
ax2.set_xlabel('Configuration')
ax2.set_ylabel('% de réduction')
ax2.set_title('📉 Pourcentage de réduction du vocabulaire')
ax2.set_xticklabels(configs_noms, rotation=45, ha='right')
ax2.grid(True, alpha=0.3)

# Ajouter les valeurs sur les barres
for bar, pct in zip(bars, pourcentages):
    if pct > 0:
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                f'{pct}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 **OBSERVATIONS :**")
print(f"• La configuration complète réduit le vocabulaire de {df_resultats.loc[df_resultats['Configuration'] == 'Complète', '% Réduction'].iloc[0]}%")
print(f"• La normalisation web seule a un impact de {df_resultats.loc[df_resultats['Configuration'] == 'Web seulement', '% Réduction'].iloc[0]}%")
print("• Chaque type de normalisation contribue différemment selon le contenu du texte")

---

# 💡 9. Conseils et Bonnes Pratiques

## ✅ Quand Utiliser la Normalisation d'Entités

| **Cas d'usage** | **Recommandation** | **Raison** |
|---|---|---|
| **Classification de documents** | ✅ Recommandé | Réduit le bruit, améliore les performances |
| **Analyse de sentiment** | ✅ Recommandé | Focus sur les mots porteurs d'émotion |
| **Recherche d'information** | ✅ Recommandé | Uniformise les requêtes |
| **Anonymisation** | ✅ **Essentiel** | Protection des données personnelles |
| **Traduction automatique** | ❌ Déconseillé | Peut perdre des informations importantes |
| **Génération de texte** | ❌ Déconseillé | Rend le texte moins naturel |
| **Analyse syntaxique** | ❌ Déconseillé | Structure grammaticale importante |

## 🎯 Stratégies par Domaine

### 💼 **E-commerce**
```python
# Garder certains montants pour l'analyse des prix
normalisateur = NormalisateurComplet(
    normaliser_montants=False,  # Garder les prix
    normaliser_web=True,        # Nettoyer les URLs
    normaliser_telephones=True  # Anonymiser
)
```

### 🏥 **Médical**
```python
# Attention aux dates et dosages
normalisateur = NormalisateurComplet(
    normaliser_dates=False,     # Dates importantes
    normaliser_nombres=False,   # Dosages cruciaux
    normaliser_web=True
)
```

### 📰 **Médias/Presse**
```python
# Anonymisation tout en gardant l'information
normalisateur = NormalisateurComplet(
    normaliser_web=True,        # URLs de sources
    normaliser_telephones=True, # Protection vie privée
    normaliser_montants=True    # Uniformiser économie
)
```

In [ ]:
# 🎯 Exercice final : Créez votre normalisateur personnalisé

def creer_normalisateur_personnalise(domaine: str):
    """
    Crée un normalisateur adapté à un domaine spécifique
    """
    configs_domaines = {
        'ecommerce': {
            'dates': True, 'montants': False, 'web': True, 
            'telephones': True, 'nombres': True
        },
        'medical': {
            'dates': False, 'montants': True, 'web': True, 
            'telephones': True, 'nombres': False
        },
        'finance': {
            'dates': True, 'montants': False, 'web': True, 
            'telephones': True, 'nombres': False
        },
        'media': {
            'dates': True, 'montants': True, 'web': True, 
            'telephones': True, 'nombres': True
        },
        'general': {
            'dates': True, 'montants': True, 'web': True, 
            'telephones': True, 'nombres': True
        }
    }
    
    config = configs_domaines.get(domaine, configs_domaines['general'])
    
    return NormalisateurComplet(
        normaliser_dates=config['dates'],
        normaliser_montants=config['montants'],
        normaliser_web=config['web'],
        normaliser_telephones=config['telephones'],
        normaliser_nombres=config['nombres']
    )

# Test des normalisateurs spécialisés
domaines = ['ecommerce', 'medical', 'finance', 'media']
texte_test = "Consultation le 15/03/2024. Coût : 50€. Contact : 01.23.45.67.89. Voir www.hopital.fr"

print("🎯 NORMALISATEURS SPÉCIALISÉS PAR DOMAINE")
print("=" * 60)
print(f"📝 Texte : {texte_test}\n")

for domaine in domaines:
    norm = creer_normalisateur_personnalise(domaine)
    resultat = norm.normaliser_texte(texte_test)
    print(f"🏷️  **{domaine.capitalize()}** : {resultat}")

print("\n💡 Notez les différences selon le domaine d'application !")

---

# 🎉 Conclusion

## 🏆 Ce que Vous Avez Appris

✅ **Normalisation des dates** : Formats français → tokens uniformes  
✅ **Normalisation des montants** : Devises et formats → MONTANT  
✅ **Normalisation web** : URLs, emails, mentions → tokens spécialisés  
✅ **Normalisation téléphones** : Formats français → TELEPHONE  
✅ **Normalisation nombres** : Chiffres et lettres → NOMBRE  
✅ **Pipeline complet** : Combinaison modulaire et configurable  
✅ **Stratégies par domaine** : Adaptation selon le contexte  

## 🚀 Prochaines Étapes

1. **Intégrez** ces techniques dans votre pipeline de preprocessing
2. **Testez** sur vos propres données
3. **Adaptez** les configurations selon votre domaine
4. **Mesurez** l'impact sur vos modèles NLP

## 📚 Pour Aller Plus Loin

- **Module suivant** : Tokenisation avancée avec spaCy
- **Projet pratique** : Pipeline complet de preprocessing
- **Ressources** : Documentation regex Python, spaCy french model

---

**💡 Astuce finale :** La normalisation d'entités est un art autant qu'une science. Testez toujours l'impact sur vos métriques finales !

**🔗 Notebook suivant :** `pipeline_complet.ipynb` - Intégration de toutes les techniques de preprocessing